In [1]:
import pandas as pd
import numpy as np 
from matplotlib import pyplot as plt

In [2]:
import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/datacamp-optimizing-online-sports-retail-revenue/reviews_v2.csv
/kaggle/input/datacamp-optimizing-online-sports-retail-revenue/finance.csv
/kaggle/input/datacamp-optimizing-online-sports-retail-revenue/brands_v2.csv
/kaggle/input/datacamp-optimizing-online-sports-retail-revenue/traffic_v3.csv
/kaggle/input/datacamp-optimizing-online-sports-retail-revenue/info_v2.csv


In [3]:
import sqlite3

brands = pd.read_csv('/kaggle/input/datacamp-optimizing-online-sports-retail-revenue/brands_v2.csv')
info = pd.read_csv('/kaggle/input/datacamp-optimizing-online-sports-retail-revenue/info_v2.csv')
finance = pd.read_csv('/kaggle/input/datacamp-optimizing-online-sports-retail-revenue/finance.csv')
traffic = pd.read_csv('/kaggle/input/datacamp-optimizing-online-sports-retail-revenue/traffic_v3.csv')
reviews = pd.read_csv('/kaggle/input/datacamp-optimizing-online-sports-retail-revenue/reviews_v2.csv')

connect = sqlite3.connect(':memory:')

In [4]:
brands.to_sql('brands', connect, index=False)
info.to_sql('info', connect, index=False)
finance.to_sql('finance', connect, index=False)
traffic.to_sql('traffic', connect, index=False)
reviews.to_sql('reviews', connect, index=False)

3179

In [5]:
query1 = """
SELECT * FROM BRANDS;
"""
Q1 = pd.read_sql(query1, connect)
Q1

,product_id,brand
0,AH2430,None
1,G27341,Adidas
2,CM0081,Adidas
3,B44832,Adidas
4,D98205,Adidas
...,...,...
3174,AT6100-606,Nike
3175,CT9155-063,Nike
3176,CI1236-100,Nike
3177,AH6799-300,Nike


In [6]:
query2 = """
SELECT COUNT (DISTINCT PRODUCT_ID) AS PROD_ID, COUNT (DISTINCT BRAND) AS BRAND FROM BRANDS;
"""
Q2 = pd.read_sql(query2, connect)
Q2

,PROD_ID,BRAND
0,3179,2


In [7]:
query3 = """
SELECT B.BRAND AS BRAND  ,  SUM(F.LISTING_PRICE) AS LISTING_PRICE , SUM(F.SALE_PRICE) AS SALES_PRICE , SUM(F.DISCOUNT) AS DISCOUNT , SUM(F.REVENUE) AS REVENUE
FROM FINANCE F JOIN BRANDS B ON F.PRODUCT_ID = B.PRODUCT_ID
WHERE B.BRAND IS NOT NULL
GROUP BY B.BRAND;
"""
Q3 = pd.read_sql(query3, connect)
Q3

,BRAND,LISTING_PRICE,SALES_PRICE,DISCOUNT,REVENUE
0,Adidas,194992.25,131835.65,861.4,11526619.08
1,Nike,22532.45,55685.82,0.0,802283.26


In [8]:
query4 = """
SELECT 
    brand, 
    COUNT(B.product_id) AS ProductsPerBrand,
    AVG(CAST(listing_price AS decimal(5,2))) AS AvgPrice,
    MIN(listing_price) AS LowestPrice,
    MAX(listing_price) AS HighestPrice
FROM brands B
JOIN finance F ON B.product_id = F.product_id
WHERE listing_price > 0 AND brand IS NOT NULL
GROUP BY brand
ORDER BY brand;
"""

Q4 = pd.read_sql(query4, connect)
Q4

,brand,ProductsPerBrand,AvgPrice,LowestPrice,HighestPrice
0,Adidas,2575,75.725146,8.99,299.99
1,Nike,191,117.970942,29.95,199.95


In [9]:
query5 = """
SELECT
    brand,
    sale_price,
    CASE
        WHEN sale_price BETWEEN 0 AND 50 THEN 'Low'
        WHEN sale_price BETWEEN 51 AND 100 THEN 'Medium'
        WHEN sale_price BETWEEN 101 AND 250 THEN 'High'
        ELSE 'Very High'
    END AS price_range_label
FROM finance F
JOIN brands B ON F.product_id = B.product_id
WHERE brand IS NOT NULL
ORDER BY 2 ASC;
"""

Q5 = pd.read_sql(query5, connect)
Q5

,brand,sale_price,price_range_label
0,Adidas,4.49,Low
1,Adidas,4.99,Low
2,Adidas,4.99,Low
3,Adidas,4.99,Low
4,Adidas,4.99,Low
...,...,...,...
3115,Adidas,279.99,Very High
3116,Adidas,279.99,Very High
3117,Adidas,299.99,Very High
3118,Adidas,299.99,Very High


In [10]:
query6 = """
SELECT B.BRAND , ROUND(AVG(F.DISCOUNT) * 100,2) AS AVG_DISCOUNT FROM BRANDS B JOIN FINANCE F ON B.PRODUCT_ID = F.PRODUCT_ID
WHERE B.BRAND IS NOT NULL
GROUP BY B.BRAND;
"""

Q6 = pd.read_sql(query6, connect)
Q6

,brand,AVG_DISCOUNT
0,Adidas,33.45
1,Nike,0.00


In [11]:
query7 = """
SELECT ROUND((COUNT(*) * SUM(R.REVIEWS  *  F.REVENUE) - SUM(R.REVIEWS) * SUM(F.REVENUE)) / (SQRT(COUNT(*) * SUM(POWER(R.REVIEWS, 2)) - POWER(SUM(R.REVIEWS), 2)) * SQRT(COUNT(*) * SUM(POWER(F.REVENUE, 2)) - POWER(SUM(F.REVENUE), 2))),2)
AS CORR_REL
FROM REVIEWS R
JOIN FINANCE F ON R.product_id = F.product_id
JOIN BRANDS B ON B.product_id = F.product_id
WHERE BRAND IS NOT NULL;
"""

Q7 = pd.read_sql(query7, connect)
Q7

,CORR_REL
0,0.65


In [12]:
query8 = """
SELECT 
    CAST(CEILING(LENGTH(description)/100.0) AS int) AS Bins,
    CAST(AVG(rating) AS float64) AS AvgRating
FROM info I
JOIN reviews R ON I.product_id = R.product_id
WHERE description IS NOT NULL
GROUP BY CEILING(LENGTH(description)/100.0)
ORDER BY CEILING(LENGTH(description)/100.0)

"""

Q8 = pd.read_sql(query8, connect)
Q8

,Bins,AvgRating
0,1,2.257143
1,2,3.185551
2,3,3.277815
3,4,3.285407
4,5,3.345763
5,6,3.120000
6,7,3.653333


In [71]:
query9 = """
SELECT B.BRAND , strftime('%m' , T.LAST_VISITED) AS MONTH ,  SUM(REVIEWS) AS COUNT_REVIEWS , ROUND(AVG(R.RATING),2) AS AVG_RATINGBYMONTH , SUM(F.REVENUE) AS MONTHLY_REVENUE FROM REVIEWS R
JOIN FINANCE F ON R.PRODUCT_ID = F.PRODUCT_ID
JOIN TRAFFIC T ON R.PRODUCT_ID = T.PRODUCT_ID
JOIN BRANDS B ON R.PRODUCT_ID = B.PRODUCT_ID
WHERE B.BRAND IS NOT NULL AND MONTH IS NOT NULL
GROUP BY B.BRAND ,strftime('%m', T.LAST_VISITED);
"""

Q9 = pd.read_sql(query9, connect)
Q9

,brand,MONTH,COUNT_REVIEWS,AVG_RATINGBYMONTH,MONTHLY_REVENUE
0,Adidas,01,11852.0,3.32,1085801.88
1,Adidas,02,13372.0,3.34,1153606.20
2,Adidas,03,12987.0,3.38,1160076.93
3,Adidas,04,9145.0,3.36,854969.43
4,Adidas,05,7358.0,3.44,708602.14
5,Adidas,06,7773.0,3.46,727417.50
6,Adidas,07,8564.0,3.33,779534.42
7,Adidas,08,9813.0,3.39,889346.96
8,Adidas,09,9246.0,3.43,879667.36
9,Adidas,10,10095.0,3.39,908125.19


In [14]:
query10 = """
WITH FOOTWEAR AS
(
SELECT B.BRAND , ROUND(AVG(F.REVENUE),2) AS AVG_REVENUE FROM BRANDS B
JOIN FINANCE F ON B.PRODUCT_ID = F.PRODUCT_ID
WHERE B.BRAND IS NOT NULL
GROUP BY B.BRAND
)

SELECT * FROM FOOTWEAR;
"""

Q10 = pd.read_sql(query10, connect)
Q10

,BRAND,AVG_REVENUE
0,Adidas,4476.36
1,Nike,1472.08


In [15]:
query11 = """
WITH footwear AS(
SELECT *
FROM info
WHERE description LIKE '%shoe%'
)
SELECT COUNT(*) AS ProductCount, AVG(CAST(revenue AS money)) AS AvgRevenue
FROM footwear FW
JOIN finance F ON FW.product_id = F.product_id
"""
Q11 = pd.read_sql(query11, connect)
Q11

,ProductCount,AvgRevenue
0,2320,4531.442668


In [23]:
query12 = """
WITH footwear AS(
SELECT *
FROM info
WHERE description LIKE '%shoe%'
)
SELECT COUNT(*) AS ProductCount, AVG(CAST(revenue AS money)) AS AvgRevenue
FROM info I
JOIN brands B ON B.product_id = I.product_id
JOIN finance F ON F.product_id = I.product_id
LEFT JOIN footwear FW ON FW.product_id = I.product_id
WHERE FW.product_id IS NULL AND brand IS NOT NULL;
"""

Q12 = pd.read_sql(query12, connect)
Q12

,ProductCount,AvgRevenue
0,800,2269.944188


In [20]:
query13 = """
SELECT I.product_name, COUNT(T.last_visited) AS visit_count, MAX(T.last_visited) AS last_visit
FROM traffic T
JOIN info I ON T.product_id = I.product_id
WHERE I.PRODUCT_NAME IS NOT NULL
GROUP BY I.product_name
ORDER BY visit_count DESC;
"""

Q13 = pd.read_sql(query13, connect)
Q13

,product_name,visit_count,last_visit
0,Men's adidas Originals Supercourt Shoes,15,2020-02-18 15:27:00
1,Women's adidas Originals Sambarose Shoes,14,2019-10-27 12:47:00
2,Men's adidas Originals Continental 80 Shoes,14,2020-02-03 16:00:00
3,Unisex adidas Originals Marathon Tech Shoes,13,2020-04-11 10:05:00
4,Nike Air Max 90,13,2020-04-12 13:44:00
...,...,...,...
1506,MEN'S ADIDAS CLOUDFOAM RACER TR SHOES,0,None
1507,LeBron x John Elliott Icon,0,None
1508,Jordan React Havoc SE,0,None
1509,Jordan 'Why Not?' Zer0.2 SE,0,None


In [28]:
query14 = """
WITH L_V AS (
SELECT I.product_name, COUNT(T.last_visited) AS visit_count, MAX(T.last_visited) AS last_visit
FROM traffic T
JOIN info I ON T.product_id = I.product_id
WHERE I.PRODUCT_NAME IS NOT NULL
GROUP BY I.product_name
ORDER BY visit_count DESC
)

SELECT COUNT(PRODUCT_NAME)  AS PRODUCT_VISITED FROM L_V
WHERE LAST_VISIT IS NOT NULL;

"""

Q14 = pd.read_sql(query14, connect)
Q14

,PRODUCT_VISITED
0,1445


In [31]:
query15 = """
WITH L_V AS (
SELECT I.product_name, COUNT(T.last_visited) AS visit_count, MAX(T.last_visited) AS last_visit
FROM traffic T
JOIN info I ON T.product_id = I.product_id
WHERE I.PRODUCT_NAME IS NOT NULL
GROUP BY I.product_name
ORDER BY visit_count DESC
)

SELECT COUNT(PRODUCT_NAME) AS PRODUCT_NOTVISITED FROM L_V
WHERE LAST_VISIT IS NULL;
"""

Q15 = pd.read_sql(query15, connect)
Q15

,PRODUCT_NOTVISITED
0,66


In [64]:
query16 = """
WITH L_V AS (
SELECT I.product_name, COUNT(T.last_visited) AS visit_count, MAX(T.last_visited) AS last_visit
FROM traffic T
JOIN info I ON T.product_id = I.product_id
WHERE I.PRODUCT_NAME IS NOT NULL
GROUP BY I.product_name
ORDER BY visit_count DESC
)

SELECT F.PRODUCT_ID , F.REVENUE , R.RATING , R.REVIEWS FROM FINANCE F , L_V
JOIN REVIEWS R ON F.PRODUCT_ID = R.PRODUCT_ID 
WHERE VISIT_COUNT > 1 AND R.RATING IS NOT NULL
GROUP BY 2
ORDER BY 2 DESC;
"""

Q16 = pd.read_sql(query16, connect)
Q16

,product_id,revenue,rating,reviews
0,310805-137,64203.93,4.7,223.0
1,FV7826,37150.45,2.4,86.0
2,FV6794,34990.54,4.1,81.0
3,EG5185,33838.31,3.9,94.0
4,EF9623,31246.88,3.1,62.0
...,...,...,...,...
2197,CL7308,32.38,2.4,1.0
2198,CM0093,32.36,3.7,2.0
2199,CM0051,16.18,5.0,1.0
2200,CM0106,8.98,5.0,1.0
